In [25]:
import pandas as pd
import numpy as np
import os
path_excel = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /data phase1/XuliExcel/'
path_false = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/Phong /Company_False/'
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p_phong = f'{p}Phong /'
feature = pd.read_excel(f'{p_phong}Compare_Feature.xlsx', sheet_name = 'Balance Sheet_ Vang + Phong ')[['Feature']]
list_feature = pd.read_excel(f'{p_phong}Compare_Feature.xlsx', sheet_name = 'Income Statement_ Hiếu ')[['Feature']]
feature = pd.concat([feature, list_feature], ignore_index=True)
if (os.path.exists(path_false) == False) or (os.path.isdir(path_false) == False):
    os.mkdir(path_false)
c1 = [path_excel+'convert_standard_4/' + i for i in os.listdir(path_excel+'convert_standard_4')]
list_path_false = c1
list_com_false = (os.listdir(path_excel+'convert_standard_4'))

In [26]:
len(list_com_false)

1095

In [ ]:
list_com_false

In [28]:
list_com_rename = [i[:10] for i in list_com_false]
df_company = pd.DataFrame({'path_name':list_com_rename,
                           'path':list_path_false})
df_company = df_company.drop_duplicates(subset = 'path_name').sort_values(by = ['path_name']).reset_index(drop = True)
df_company['Symbol'] = df_company['path_name'].apply(lambda row:row[:3])
df_company['Symbol_year'] = df_company['path_name'].apply(lambda row:row[:8])
list_symbol_unique = list(df_company['Symbol'].unique())
df_company

,path_name,path,Symbol,Symbol_year
0,SJC_2014_2,/content/drive/MyDrive/Data Lake/Ingestion/Day...,SJC,SJC_2014
1,SJC_2014_3,/content/drive/MyDrive/Data Lake/Ingestion/Day...,SJC,SJC_2014
2,SJD_2009_1,/content/drive/MyDrive/Data Lake/Ingestion/Day...,SJD,SJD_2009
3,SJD_2009_2,/content/drive/MyDrive/Data Lake/Ingestion/Day...,SJD,SJD_2009
4,SJD_2009_3,/content/drive/MyDrive/Data Lake/Ingestion/Day...,SJD,SJD_2009
...,...,...,...,...
1090,XMC_2021_4,/content/drive/MyDrive/Data Lake/Ingestion/Day...,XMC,XMC_2021
1091,XMC_2021_5,/content/drive/MyDrive/Data Lake/Ingestion/Day...,XMC,XMC_2021
1092,XPH_2021_1,/content/drive/MyDrive/Data Lake/Ingestion/Day...,XPH,XPH_2021
1093,XPH_2021_2,/content/drive/MyDrive/Data Lake/Ingestion/Day...,XPH,XPH_2021


In [29]:
def drop_col__(df):
    list_year_drop = []
    for i in df.columns:
        if '__' in i:
            list_year_drop.append(i)
    df = df.drop(columns = list_year_drop)
    return df

In [30]:
def merge_company(symbol):
    df = feature.copy()
    list_symbol_year = df_company[df_company['Symbol'] == symbol]['Symbol_year'].unique()
    for symbol_year in list_symbol_year:
        # print(symbol_year)
        df_symbol = df_company[df_company['Symbol_year'] == symbol_year].reset_index(drop = True)
        df_year = pd.DataFrame()
        for i in df_symbol.index:
            df_tam = pd.read_csv(df_symbol['path'].iloc[i]).rename(columns = {'feature':'Feature'})
            df_tam = df_tam.drop(columns = ['code'])
            df_year = pd.concat([df_year, df_tam], ignore_index=True)
        df = pd.merge(df, df_year, how = 'left', on = ['Feature'], suffixes = ('', '__'))
    return drop_col__(df)
symbol = 'MAS'
df = merge_company(symbol)
df

,Feature
0,CURRENT ASSETS
1,Cash and cash equivalents
2,Cash
3,Cash equivalents
4,Short Term Financial Investments
...,...
135,Profit after tax
136,Net profit after tax of the parent
137,Benefits of minority shareholders
138,Basic earnings per share


In [33]:
len(os.listdir(path_false))

932

In [ ]:
for i in range(len(list_symbol_unique)):
    symbol = list_symbol_unique[i]
    print(symbol)
    merge_company(symbol).to_csv(f'{path_false}{symbol}.csv', index = False)

In [ ]:
for symbol in os.listdir(path_false):
    os.remove(f'{path_false}{symbol}')